# Causal Estimation - ATE and CATE

This notebook estimates:
1. Propensity scores
2. Average Treatment Effects (ATE) using multiple methods
3. Conditional Average Treatment Effects (CATE) for heterogeneity

In [ ]:
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from coupon_causal import data, features, propensity, ate, cate, balance, viz, utils

%matplotlib inline

## 1. Load Data and Prepare Features

In [ ]:
config = utils.load_config('../config/default.yaml')
utils.set_random_seed(config['random_state'])

df, ground_truth = data.generate_synthetic_coupon_data(
    n_samples=config['synthetic']['n_samples'],
    treatment_rate=config['synthetic']['treatment_rate'],
    true_ate=config['synthetic']['true_ate'],
    random_state=config['random_state']
)

print(f"True ATE: ${ground_truth['true_ate']:.2f}")

In [ ]:
# Prepare features
X, T, Y, feature_engineer = features.prepare_features(df, config['features'], fit=True)

print(f"Feature matrix shape: {X.shape}")
print(f"Treatment rate: {T.mean():.1%}")
print(f"Mean outcome: ${Y.mean():.2f}")

## 2. Propensity Score Modeling

In [ ]:
# Fit propensity models
logistic_prop, gbm_prop, ensemble_prop = propensity.fit_propensity_models(X, T, config)

# Diagnostics
diagnostics = logistic_prop.compute_diagnostics(T)
print("\nPropensity Model Diagnostics:")
for key, value in diagnostics.items():
    print(f"  {key}: {value:.3f}")

In [ ]:
# Visualize propensity scores
viz.plot_propensity_distribution(ensemble_prop, T)
plt.show()

In [ ]:
# Check overlap
overlap_metrics = propensity.assess_overlap(ensemble_prop, T)
print("\nOverlap Assessment:")
for key, value in overlap_metrics.items():
    print(f"  {key}: {value}")

## 3. Balance Diagnostics

In [ ]:
# Compute IPW weights
ipw_weights = propensity.compute_ipw_weights(
    T, ensemble_prop, 
    stabilize=True, 
    trim_percentiles=tuple(config['propensity']['trim_percentiles'])
)

# Check balance before and after
balance_before, balance_after, balance_summary = balance.compare_balance_before_after(
    X, T, feature_engineer.get_feature_names(), ipw_weights
)

In [ ]:
# Love plot
viz.plot_love_plot(balance_before, balance_after, top_k=15)
plt.show()

## 4. Average Treatment Effect (ATE) Estimation

In [ ]:
# Estimate ATE using all methods
ate_results = ate.estimate_all_ate_methods(X, Y, T, ensemble_prop, config)

In [ ]:
# Display results
ate_summary = pd.DataFrame([
    {
        'Method': method.replace('_', ' ').title(),
        'ATE': f"${res['ate']:.2f}",
        '95% CI': f"[${res['ci_lower']:.2f}, ${res['ci_upper']:.2f}]",
        'Error vs True': f"${res['ate'] - ground_truth['true_ate']:.2f}"
    }
    for method, res in ate_results.items()
])

print(f"\nTrue ATE: ${ground_truth['true_ate']:.2f}")
print("\nATE Estimates:")
print(ate_summary.to_string(index=False))

In [ ]:
# Visualize ATE comparison
viz.plot_ate_comparison(ate_results)
plt.axvline(ground_truth['true_ate'], color='green', linestyle=':', linewidth=2, label='True ATE')
plt.legend()
plt.show()

## 5. Conditional Average Treatment Effect (CATE) Estimation

In [ ]:
# Estimate CATE using all methods
cate_results = cate.estimate_all_cate_methods(
    X, Y, T, ensemble_prop, config, feature_engineer.get_feature_names()
)

In [ ]:
# Display CATE summary
cate_summary = pd.DataFrame([
    {
        'Method': method,
        'Mean CATE': f"${res['mean']:.2f}",
        'Std CATE': f"${res['std']:.2f}",
        'Range': f"[${res['min']:.2f}, ${res['max']:.2f}]"
    }
    for method, res in cate_results.items() if res is not None
])

print("\nCATE Estimates:")
print(cate_summary.to_string(index=False))
print(f"\nTrue CATE std: ${ground_truth['cate_std']:.2f}")

In [ ]:
# Use Orthogonal Forest CATE (if available)
if cate_results.get('orthogonal_forest') is not None:
    cate_scores = cate_results['orthogonal_forest']['cate']
    cate_method = 'Orthogonal Forest'
elif cate_results.get('dr_learner') is not None:
    cate_scores = cate_results['dr_learner']['cate']
    cate_method = 'DR Learner'
else:
    cate_scores = cate_results['x_learner']['cate']
    cate_method = 'X-Learner'

print(f"\nUsing {cate_method} for analysis")

In [ ]:
# Visualize CATE distribution
viz.plot_cate_distribution(cate_scores, method_name=cate_method)
plt.show()

In [ ]:
# Compare predicted vs true CATE (for synthetic data)
if 'true_treatment_effect' in df.columns:
    true_cate = df['true_treatment_effect'].values
    
    plt.figure(figsize=(10, 6))
    plt.scatter(true_cate, cate_scores, alpha=0.3, s=10)
    plt.plot([true_cate.min(), true_cate.max()], 
             [true_cate.min(), true_cate.max()], 
             'r--', linewidth=2, label='Perfect prediction')
    plt.xlabel('True Treatment Effect', fontsize=12)
    plt.ylabel('Predicted Treatment Effect', fontsize=12)
    plt.title(f'CATE Prediction Quality - {cate_method}', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Compute correlation
    corr = np.corrcoef(true_cate, cate_scores)[0, 1]
    plt.text(0.05, 0.95, f'Correlation: {corr:.3f}', 
             transform=plt.gca().transAxes, 
             fontsize=12,
             verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.show()

## Summary

Key findings:
1. **Propensity model**: Good overlap and calibration
2. **Balance**: IPW substantially improves covariate balance
3. **ATE**: Doubly-robust methods (AIPW) recover true ATE well
4. **CATE**: Substantial heterogeneity in treatment effects

Next: Proceed to policy uplift notebook (20_policy_uplift.ipynb) to evaluate targeting strategies